# Assignment 5 : Build a TFIDF Vectorizer

## Task-1 : Build a TFIDF Vectorizer & compare its results with Sklearn.

### Corpus

In [141]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [142]:
""""https://stackoverflow.com/questions/60042735/
how-to-build-a-tfidf-vectorizer-given-a-corpus-and-compare-its-results-using-skl"""

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math as m
import operator 
from sklearn.preprocessing import normalize
import numpy

In [143]:
""""https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.
TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer.transform"""
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
vectorizer.fit(corpus)
skl_output=vectorizer.transform(corpus)

FIT FUNCTION FOR TFIDF

In [144]:
def fit(dataset):
    vocab=set()
    #https://www.w3schools.com/python/ref_func_isinstance.asp
    if isinstance(dataset,(list)):
        for row in dataset:
            for word in row.split(" "):
                if len(word) < 2:
                    continue
                vocab.add(word)
        unique_words= sorted(list(vocab))
        #https://www.geeksforgeeks.org/enumerate-in-python/
        vocab = {j:i for i,j in enumerate(unique_words)}
        return vocab
        print("you need to pass list of sentance")

In [145]:
vocab=fit(corpus)
print(vocab)

{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}


In [146]:
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


* **BOTH THE VOCABULARY ARE SAME**

## TERM FREQUENCY(TF)

In [147]:
def tf(corpus,vocab):
    tf={}
    if isinstance(corpus,(list)):
        for idx,row in enumerate(corpus):
            #https://www.geeksforgeeks.org/python-counter-objects-elements/
            word_freq=dict(Counter(row.split()))
            c=sum(word_freq.values())
            """"https://www.w3schools.com/python/ref_dictionary_values
            .asp#:~:text=The%20values()%20method%20returns,the%20dictionary%2C%20see%20example%20below."""
            for act in vocab.keys():
                if len(act) < 2:
                        continue
                if act in word_freq:
                    tf[act]=word_freq[act]/c
                else:
                    tf[act]=0  
    return(tf)   

tf(corpus,vocab)                

{'and': 0,
 'document': 0.2,
 'first': 0.2,
 'is': 0.2,
 'one': 0,
 'second': 0,
 'the': 0.2,
 'third': 0,
 'this': 0.2}

## IDF: Inverse Document Frequency

In [148]:
def idf(corpus,vocab):
    idf={}
    N=len(corpus)
   
    for vky in vocab.keys():
        n=0
        for idx, row in enumerate((corpus)):
            if len(vky) < 2:
                continue
            if vky in row:
                n=n+1             
        idf[vky]=1+(m.log((1+N)/(1+float(n))))
        
    return(idf) 

idf(corpus,vocab)                

{'and': 1.916290731874155,
 'document': 1.2231435513142097,
 'first': 1.5108256237659907,
 'is': 1.0,
 'one': 1.916290731874155,
 'second': 1.916290731874155,
 'the': 1.0,
 'third': 1.916290731874155,
 'this': 1.0}

In [149]:
print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


Both idf are same value

In [150]:
idfv=idf(corpus,vocab)

### TRANSFORM FUNCTION :

In [151]:
def tranform(corpus,idfs):
    rows=[]
    colums=[]
    values=[]
    tfidf={}
    if isinstance(corpus, (list,)):
        for idx, row in enumerate(tqdm(corpus)):
            lis=[]
            lis.append(row)
            tfval=tf(lis,vocab) 
            for word,value in idfs.items():
                if len(word) < 2:
                    continue
                tfidf[word]=idfv[word]*tfval[word]
                
                if tfidf[word]!=0:
                    rows.append(idx)
                    colums.append(value)
                    values.append(tfidf[word])
                l=csr_matrix((values, (rows,colums)), shape=(len(corpus),len(vocab)))# creating the sparse matrix
                k=normalize(l,norm='l2') # normalize the sparse matrix(l2 norm)
        print(k[0])
        print()
        print(k[0].toarray()) #converting intro dense matrix
        print("shape of sparse matrix",k.shape)     
    

In [152]:
#TFIDF VALUES OF FIRST DOCUMENT STORED IN SPARSE MATRIX AND CONVERTED INTO DENSE MATRIX
tranform(corpus,vocab)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 322.89it/s]

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]
shape of sparse matrix (4, 9)


### SkLearn Implementation

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [154]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [155]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [156]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [157]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [158]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Your custom implementation

In [159]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

## Task-2

<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task you have to modify your fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
   </ul>

In [160]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


**Creating vocabulary of unique words from corpus**

In [161]:
def fit(dataset):
    vocab=set()
    #https://www.w3schools.com/python/ref_func_isinstance.asp
    if isinstance(dataset,(list)):
        for row in dataset:
            for word in row.split(" "):
                if len(word) < 2:
                    continue
                vocab.add(word)
        unique_words= sorted(list(vocab))
        #https://www.geeksforgeeks.org/enumerate-in-python/
        vocab = {j:i for i,j in enumerate(unique_words)}
        return vocab
        print("you need to pass list of sentance")

In [162]:
vocab=fit(corpus)

## IDF

In [163]:
def idf(corpus,vocab):
    idf={}
    N=len(corpus)
    idfsorted={}
    idf50={}
    vocab50={}
    i=0
   
    for vky in vocab.keys():
        n=0
        for idx, row in enumerate((corpus)):
            if len(vky) < 2:
                continue
            if vky in row:
                n=n+1             
        idf[vky]=1+(m.log((1+N)/(1+float(n))))
        
    for k in sorted(idf,key=idf.get,reverse=True):
        idfsorted[k]=idf[k]
        z=Counter(idfsorted)
        top=z.most_common(50)
    for a,b in top:
        vocab50[a]=i       #added word in vocab and a number value
        idf50[a]=b         #added idf value in idf50
        i=i+1              #increase number for each word in vocab
    return vocab50,idf50    
        

In [164]:
vocab50,idf50=idf(corpus,vocab)

In [165]:
print('Top 50 idf value and associated words')
print("\n")
print(idf50)

Top 50 idf value and associated words


{'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.922918004572872, 'abstruse': 6.922918004572872, 'academy': 6.922918004572872, 'accents': 6.922918004572872, 'accessible': 6.922918004572872, 'acclaimed': 6.922918004572872, 'accolades': 6.922918004572872, 'accurately': 6.922918004572872, 'achille': 6.922918004572872, 'ackerman': 6.922918004572872, 'adams': 6.922918004572872, 'added': 6.922918004572872, 'admins': 6.922918004572872, 'admiration': 6.922918004572872, 'admitted': 6.922918004572872, 'adrift': 6.922918004572872, 'adventure': 6.922918004572872, 'aesthetically': 6.922918004572872, 'affected': 6.922918004572872, 'affleck': 6.922918004572872, 'afternoon': 6.922918004572872, 'agreed': 6.922918004572872, 'aimless': 6.922918004572872, 'aired': 6.922918004572872, 'akasha': 6.922918004572872, 'alert': 6.922918004572872, 'alike': 6.922918004572872, 'allison': 6.922918004572872, 'allowing': 6.922918004572872, 'alongside': 

In [166]:
print('50 words of top idf values')
print("\n")
print(vocab50)

50 words of top idf values


{'aailiyah': 0, 'abandoned': 1, 'abroad': 2, 'abstruse': 3, 'academy': 4, 'accents': 5, 'accessible': 6, 'acclaimed': 7, 'accolades': 8, 'accurately': 9, 'achille': 10, 'ackerman': 11, 'adams': 12, 'added': 13, 'admins': 14, 'admiration': 15, 'admitted': 16, 'adrift': 17, 'adventure': 18, 'aesthetically': 19, 'affected': 20, 'affleck': 21, 'afternoon': 22, 'agreed': 23, 'aimless': 24, 'aired': 25, 'akasha': 26, 'alert': 27, 'alike': 28, 'allison': 29, 'allowing': 30, 'alongside': 31, 'amateurish': 32, 'amazed': 33, 'amazingly': 34, 'amusing': 35, 'amust': 36, 'anatomist': 37, 'angela': 38, 'angelina': 39, 'angry': 40, 'anguish': 41, 'angus': 42, 'animals': 43, 'animated': 44, 'anita': 45, 'anniversary': 46, 'anthony': 47, 'antithesis': 48, 'anyway': 49}


## TERM FREQUENCY(TF)

In [167]:
def tf(corpus,vocab):
    tf={}
    if isinstance(corpus,(list)):
        for idx,row in enumerate(corpus):
            #https://www.geeksforgeeks.org/python-counter-objects-elements/
            word_freq=dict(Counter(row.split()))
            c=sum(word_freq.values())
            """"https://www.w3schools.com/python/ref_dictionary_values
            .asp#:~:text=The%20values()%20method%20returns,the%20dictionary%2C%20see%20example%20below."""
            for act in vocab.keys():
                if len(act) < 2:
                        continue
                if act in word_freq:
                    tf[act]=word_freq[act]/c
                else:
                    tf[act]=0  
    return(tf)   

### TRANSFORM FUNCTION

In [168]:
def tranform(corpus,vocab):
    rows=[]
    colums=[]
    values=[]
    tfidf={}
    if isinstance(corpus, (list,)):
        for idx, row in enumerate(tqdm(corpus)):
            lis=[]
            lis.append(row) # for each document we are calling the function tf to calculate tf values for a particular document
            
            tfval=tf(lis,vocab50) 
            for word,value in idf50.items():
                if len(word) < 2:
                    continue
                tfidf[word]=idf50[word]*tfval[word]
                
                if tfidf[word]!=0:
                    rows.append(idx)
                    colums.append(value)
                    values.append(tfidf[word])
                l=csr_matrix((values, (rows,colums)), shape=(len(corpus),len(vocab50)))# creating the sparse matrix
                k=normalize(l,norm='l2',) # normalize the sparse matrix(l2 norm)
        print(k[0])
        print()
        print(k[0].toarray()) #converting intro dense matrix
        print("shape of sparse matrix",k.shape) 

In [169]:
tranform(corpus,vocab) 

100%|████████████████████████████████████████████████████████████████████████████████| 746/746 [00:11<00:00, 67.44it/s]

  (0, 6)	1.0

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
shape of sparse matrix (746, 50)
